In [1]:
import torch
from torch import nn
from tests_backpropagation import main_test

torch.manual_seed(123)
torch.set_default_dtype(torch.double)


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.1151, 0.2688, 0.4053, 0.3931, 0.1285, 0.0924, 0.155, 0.0438, 0.156, 0.0461]

 --------- Comparing with autograd values  ----------- 

 ******* fc['1'].weight.grad ******* 
  Our computation:
 tensor([[-1.7814e-05, -1.7557e-05],
        [-5.6907e-05, -5.6087e-05],
        [ 1.5985e-04,  1.5754e-04]])

  Autograd's computation:
 tensor([[-1.7763e-05, -1.7507e-05],
        [-5.6887e-05, -5.6067e-05],
        [ 1.5984e-04,  1.5753e-04]])

 ********* fc['1'].bias.grad ******* 
  Our computation:
 tensor([-1.9551e-06, -6.2455e-06,  1.7543e-05])
  Autograd's computation:
 tensor([-1.9495e-06, -6.2433e-06,  1.7542e-05])

 ------------------- relative erro

## Class ``MyNet``

Read carefully how ``MyNet`` is implemented in the cell below. In particular:  
- ``n_hid`` is a list of integer, representing the number of hidden units in each hidden layer.   
-  ``MyNet([2, 3, 2]) = MiniNet()`` where ``MiniNet`` is the neural network defined in the fourth tutorial, in which notations are also clarified.     
- ``model.L`` is the number of hidden layers, ``L``   
- ``model.f[l]`` is the activation function of layer ``l``, $f^{[l]}$ (here ``torch.tanh``)   
- ``model.df[l]`` is the derivative of the activation function, $f'^{[l]}$   
- ``model.a[l]``  is the tensor $A^{[l]}$, (shape: ``(1, n(l))``)   
- ``model.z[l]``  is the tensor $Z^{[l]}$, (shape: ``(1, n(l))``)  
- Weights $W^{[l]}$ (shape: ``(n(l+1), n(l))``) and biases $\mathbf{b}^{[l]}$ (shape: ``(n(l+1))``) can be accessed as follows:
```
weights = model.fc[str(l)].weight.data
bias = model.fc[str(l)].bias.data
```

In [5]:
class MyNet(nn.Module):
    def __init__(self, n_l = [2, 3, 2]):
        super().__init__() 
        
        
        # number of layers in our network (following Andrew's notations)
        self.L = len(n_l)-1
        self.n_l = n_l
        
        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a=f(z))
        self.z = {i : None for i in range(1, self.L+1)}
        self.a = {i : None for i in range(self.L+1)}

        # Where we will store the gradients for our custom backpropagation algo
        self.dL_dw = {i : None for i in range(1, self.L+1)}
        self.dL_db = {i : None for i in range(1, self.L+1)}

        # Our activation functions
        self.f = {i : lambda x : torch.tanh(x) for i in range(1, self.L+1)}

        # Derivatives of our activation functions
        self.df = {
            i : lambda x : (1 / (torch.cosh(x)**2)) 
            for i in range(1, self.L+1)
        }
        
        # fully connected layers
        # We have to use nn.ModuleDict and to use strings as keys here to 
        # respect pytorch requirements (otherwise, the model does not learn)
        self.fc = nn.ModuleDict({str(i): None for i in range(1, self.L+1)})
        for i in range(1, self.L+1):
            self.fc[str(i)] = nn.Linear(in_features=n_l[i-1], out_features=n_l[i])
        
    def forward(self, x):
        # Input layer
        self.a[0] = torch.flatten(x, 1)
        
        # Hidden layers until output layer
        for i in range(1, self.L+1):

            # fully connected layer
            self.z[i] = self.fc[str(i)](self.a[i-1])
            # activation
            self.a[i] = self.f[i](self.z[i])

        # return output
        return self.a[self.L]

## Tasks

Write a function ``backpropagation(model, y_true, y_pred)`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[l][i,j]`` for $l \in [1 .. L]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[l][j]`` for $l \in [1 .. L]$ 

assuming ``model`` is an instance of the ``MyNet`` class.

A vectorized implementation would be appreciated.

In [56]:
def backpropagation(model, y_true, y_pred):
    with torch.no_grad():
        # MSE = (pred - true)^2
        # MSE_derived = 2(pred - true)
        # The slope of the loss function can be computed like this:
        grad_post_activation = 2 * (y_pred - y_true)

        # Compute gradient for each layer backwords from output layer to the first hidden layer
        for layer in range(model.L, 0, -1):

            # Slope of loss with respect to pre-activation of current layer
            grad_pre_activation = grad_post_activation * model.df[layer](model.z[layer])

            # Compute gradients of the loss with respect to the weights and biases of the current layer
            model.dL_dw[layer] = torch.matmul(grad_pre_activation.T, model.a[layer - 1])
            model.dL_db[layer] = torch.sum(grad_pre_activation, dim=0)

            # Compute the gradient of the loss with respect to the output of the previous layer
            grad_post_activation = torch.matmul(grad_pre_activation, model.fc[str(layer)].weight)

        learning_rate = 1e-9
        for layer in range(1, model.L + 1):
            model.fc[str(layer)].weight -= learning_rate * model.dL_dw[layer]
            model.fc[str(layer)].bias -= learning_rate * model.dL_db[layer]

## Run the cells below, and check the output

- In the 1st cell, we use a toy dataset and the same architecture as the MiniNet class of the fourth tutorial. 
- In the 2nd cell, we use a few samples of the MNIST dataset with a consistent model architecture (``24x24`` black and white cropped images as input and ``10`` output classes). 

You can set ``verbose`` to ``True`` if you want more details about your computations versus what is expected.

In [58]:
model = MyNet([2, 3, 2])
main_test(backpropagation, model, verbose=False, data='toy')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 TEST PASSED: Gradients consistent with autograd's computations.

 TEST PASSED: Gradients consistent with finite differences computations.

 __________________________________________________________________ 
                 Check that weights have been updated               
 __________________________________________________________________ 

 TEST PASSED: Weights have been updated.

 __________________________________________________________________ 
                      Check computational graph                     
 __________________________________________________________________ 

 TEST PASSED: All parameters seem correctly attached to the computational graph!

 __________________________________________________________________ 
                             Conclusion 

In [57]:
model = MyNet([24*24, 16, 10])
main_test(backpropagation, model, verbose=False, data='mnist')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 TEST PASSED: Gradients consistent with autograd's computations.

 TEST PASSED: Gradients consistent with finite differences computations.

 __________________________________________________________________ 
                 Check that weights have been updated               
 __________________________________________________________________ 

 TEST PASSED: Weights have been updated.

 __________________________________________________________________ 
                      Check computational graph                     
 __________________________________________________________________ 

 TEST PASSED: All parameters seem correctly attached to the computational graph!

 __________________________________________________________________ 
                             Conclusion 